# KDD Assignment 1
![CS306](https://img.shields.io/badge/CS306-Data%20Mining-orange) &nbsp;
![2022s](https://img.shields.io/badge/semester-2022%20spring-blue)

Author: 何泽安 (He Zean) &nbsp;&nbsp; SID: 12011323

## Step 1. Read the Data

When we simply use `pd.read_csv('HW1data.csv')` to read the data, we will get a `UnicodeDecodeError`, which means we need first to detect the actual encoding of this file (instead of the default UTF-8).
> requiring `chardet` >= 4.0.0

In [ ]:
import chardet

with open('HW1data.csv', 'rb') as f:
    enc = chardet.detect(f.read(1000))['encoding']  # give chardet 1000 bytes to let it guess is enough

print(enc)

In [ ]:
import pandas as pd

data = pd.read_csv('HW1data.csv', encoding=enc)
data.info()

Now we have successfully loaded the data with pandas, we can check if there exists any missing values.

The following result shows that for all the 14 columns, about `9.5%` of the data is missing.

In [ ]:
data.isnull().sum() / len(data)

## Step 2. Attempt to Complement the Missing Values

### (a) Extract `{PatientId-Gender-Age-Neighbourhood}`

The point is, one patient may have multiple records (check this by calling `data['PatientId'].nunique()`), therefore the `patient_base_info`, which views the PatientId as the primary key, should be `groupby` the id first.

When we are aggregating the data, we notice that the `age` of a patiend is not always consistent, so we need to use the mean value to represent it, while other two features are not numeric, and we cannot judge if there are some inconsistents easily, we tend to use `first` to aggregate them.

We thus find out that there are 62299 patients.

In [ ]:
patient_base_info = data[['PatientId', 'Gender', 'Age', 'Neighbourhood']] \
    .groupby('PatientId') \
    .agg({'Gender': 'first',             # use the first non n/a value to repr the patiend's gender
          'Age': 'mean',                 # for some patients, their age are not consistent, let's take their avg
          'Neighbourhood': 'first'})     # use the first non n/a value to repr the patiend's meighbourhood
patient_base_info.info()

### (b) Drop the Missing Values of Some Columns

Now, all the rows where any of `PatientID`, `ScheduledDay`, `AppointmentDay`, `SMS_received` or `No-show` is missed are removed.

Note that the number of columns is reduced from `667536` (see code cell 2) to `413654`.

In [ ]:
data.dropna(subset=['PatientId', 'ScheduledDay', 'AppointmentDay', 'SMS_received', 'No-show'], inplace=True)  # any record that missing any of these `key` columns contribute nothing for the further module training
data.info()

### (c) Use the Previously Extracted Info to Complete the Missing Values

In [ ]:
# we no more need the origin data, since patient_base_info has its backup
data.drop(['Gender', 'Age', 'Neighbourhood'], axis=1, inplace=True)

res = pd.merge(data, patient_base_info, on='PatientId')
res.info()

## Step 3. Another `dropna`

Actually do no effects.

In [ ]:
res.dropna(subset=['PatientId', 'Gender', 'Age', 'Neighbourhood',
           'ScheduledDay', 'AppointmentDay', 'SMS_received', 'No-show'], inplace=True)
# actually affect nothing, since there is no NaN for these cols

res.info()

## Step 4. Fill the Missing Values with Default Value `0`

In [ ]:
res.fillna({'Scholarship': 0,
            'Hipertension': 0,
            'Diabetes': 0,
            'Alcoholism': 0,
            'Handcap': 0},
           inplace=True)  # as mentioned in the docs
res.info()

## Step 5. Drop the Unused Features `PatientId` and `AppointmentID`

So far so good, now we have a nicely dataframe with no missing values!

> As the group chat mentioned, some `PatientId` are floating point numbers, but it actually does not affect the result.

In [ ]:
res.drop(['PatientId', 'AppointmentID'], axis=1, inplace=True)  # these id was for merging and cleaning the data, but contributes nothing for our module
res.info()

## Step 6. Drop the "Dirty Records"

Let's glance at the dataframe first. We can easily find a trival dissonant tone, `Age.min`! Let's throw them away.

In [ ]:
# change the Dtype from object to datetime64, let pandas better analyzing the data
res['ScheduledDay'] = pd.to_datetime(res['ScheduledDay'])
res['AppointmentDay'] = pd.to_datetime(res['AppointmentDay'])

res.describe(include='all', datetime_is_numeric=True)  # then check the table manually

In [ ]:
res.drop(res[res['Age'] < 0].index, inplace=True)  # age < 0 must be invalid
res.info()  # 4 dirty records are removed

Let's think deeper: Scholarship, Hipertension... they are all numerical features, it's really hard for us to judge if they are invalid or not. But since we just parsed `ScheduledDay` and `AppointmentDay`, why not also check them?

When a patient making an schedule online, of cause he/she could not appointes to the day before that day. Here are about 82.6% of the records violating this rule and thus need to be removed.

In [ ]:
print(len(res[res['ScheduledDay'] > res['AppointmentDay']]) / len(res))  # percentage of records containing invalid schedule day
res.drop(res[res['ScheduledDay'] > res['AppointmentDay']].index, inplace=True)  # schedule to a day before appointment is not possible
res.info()

## Step 7. Calculate the `Delta_Day`

We need a number to represent the number of days between the `ScheduledDay` and `AppointmentDay`, instead of a precise timedelta.

In [ ]:
res['Delta_Day'] = (res['AppointmentDay'] - res['ScheduledDay']).astype('timedelta64[D]')  # only take the number of days
res['Delta_Day'].describe()

## Step 8. Find the Day of Week for `ScheduledDay` and `AppointmentDay`

In [ ]:
res['SDay_DOW'] = res['ScheduledDay'].dt.dayofweek
res['ADay_DOW'] = res['AppointmentDay'].dt.dayofweek

print(res['SDay_DOW'].value_counts())  # most patients schedule on Monday
print(res['ADay_DOW'].value_counts())  # most patients appointment on Tuesday

## Step 9. Remove the Raw Dates

In [ ]:
res.drop(['ScheduledDay', 'AppointmentDay'], axis=1, inplace=True)

## Enjoy Our Dog Food

In [ ]:
res.info()

In [ ]:
res.describe(include='all', datetime_is_numeric=True)

In [ ]:
res.head(20)